<a href="https://colab.research.google.com/github/nelzonapa/cia-final-project-pro/blob/main/CEMLik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FASE 0: INGESTA Y ETL (EXTRACT, TRANSFORM, LOAD)**

## Dependencias

In [ ]:
# ==========================================
# FASE 0: INGESTA Y ETL (EXTRACT, TRANSFORM, LOAD)
# ==========================================

import os
import json
import glob
import shutil
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from PIL import Image
!pip install PyMuPDF
import fitz  # PyMuPDF para PDFs
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 63.7 MB/s eta 0:00:00


## 1. CONFIGURACIÓN DE RUTAS Y ENTORNO


In [ ]:
# 1. CONFIGURACIÓN DE RUTAS Y ENTORNO
# ------------------------------------------------------
# Montamos Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Rutas base definidas por el usuario
BASE_DATASET_PATH = "/content/drive/MyDrive/Proyecto CIA/Dataset VAST 7"
OUTPUT_PATH = "/content/drive/MyDrive/Proyecto_Tesis_Multimodal/data_procesada"
EXTRACTED_IMGS_PATH = os.path.join(OUTPUT_PATH, "imagenes_extraidas_pdf")

# Crear directorios de salida si no existen
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(EXTRACTED_IMGS_PATH, exist_ok=True)

OUTPUT_FILE = os.path.join(OUTPUT_PATH, "dataset_procesado.jsonl")

print(f"Ruta del Dataset: {BASE_DATASET_PATH}")
print(f"Ruta de Salida: {OUTPUT_FILE}")

Ruta del Dataset: /content/drive/MyDrive/Proyecto CIA/Dataset VAST 7
Ruta de Salida: /content/drive/MyDrive/Proyecto_Tesis_Multimodal/data_procesada/dataset_procesado.jsonl


## 2. FUNCIONES AUXILIARES


In [ ]:
# 2. FUNCIONES AUXILIARES
# ------------------------------------------------------

def es_imagen_valida(ruta_imagen):
    """
    Filtra imágenes decorativas (iconos, barras, gifs pequeños).
    Retorna True si la imagen es relevante semánticamente.
    """
    try:
        if not os.path.exists(ruta_imagen):
            return False

        # Filtrar por extensión (evitar gifs de sistema pequeños si es necesario)
        if ruta_imagen.lower().endswith('.gif'):
            # Regla heurística: en datasets viejos los gifs suelen ser adornos
            # Pero en VAST 2007 hay mapas o gráficos en GIF. Los validaremos por tamaño.
            pass

        with Image.open(ruta_imagen) as img:
            width, height = img.size
            # Descartar imágenes muy pequeñas (iconos, botones) < 50x50px
            if width < 50 or height < 50:
                return False

            # Descartar imágenes con relaciones de aspecto extremas (barras separadoras)
            aspect_ratio = width / height
            if aspect_ratio > 10 or aspect_ratio < 0.1:
                return False

        return True
    except Exception as e:
        # Si la imagen está corrupta, se descarta
        return False

def guardar_registro(registro, archivo_salida):
    """Escribe una línea JSON en el archivo final."""
    with open(archivo_salida, 'a', encoding='utf-8') as f:
        f.write(json.dumps(registro, ensure_ascii=False) + '\n')

# 3. MÓDULOS DE PROCESAMIENTO
# ------------------------------------------------------

def procesar_blogs(base_path):
    """
    Módulo A: Blogs (HTML + Imágenes Locales)
    Recorre carpetas, parsea HTML y asocia imágenes <img> encontradas.
    """
    print("--- Iniciando Módulo Blogs ---")
    ruta_blogs = os.path.join(base_path, "Blogs")
    contador = 0

    # Recorremos recursivamente buscando archivos .htm o .html
    for root, dirs, files in os.walk(ruta_blogs):
        for file in files:
            if file.lower().endswith(('.htm', '.html')):
                ruta_archivo = os.path.join(root, file)

                try:
                    with open(ruta_archivo, 'r', encoding='latin-1') as f: # latin-1 es común en 2007
                        soup = BeautifulSoup(f, 'html.parser')

                    # 1. Extraer Texto
                    # Eliminamos scripts y estilos para limpiar
                    for script in soup(["script", "style"]):
                        script.decompose()
                    texto = soup.get_text(separator=' ').strip()
                    texto = ' '.join(texto.split()) # Limpiar espacios extra

                    # 2. Extraer Imágenes Relacionadas
                    imagenes_asociadas = []
                    imgs = soup.find_all('img')
                    for img in imgs:
                        src = img.get('src')
                        if src:
                            # Resolver ruta relativa
                            ruta_img_absoluta = os.path.join(root, src)
                            # Normalizar ruta (quitar ../ etc)
                            ruta_img_absoluta = os.path.normpath(ruta_img_absoluta)

                            if es_imagen_valida(ruta_img_absoluta):
                                imagenes_asociadas.append(ruta_img_absoluta)

                    # Guardar solo si hay contenido relevante
                    if texto or imagenes_asociadas:
                        doc = {
                            "id": f"blog_{file}_{contador}",
                            "fuente": "Blogs",
                            "ruta_origen": ruta_archivo,
                            "texto": texto,
                            "imagenes": imagenes_asociadas,
                            "tipo": "multimodal" if imagenes_asociadas else "texto"
                        }
                        guardar_registro(doc, OUTPUT_FILE)
                        contador += 1

                except Exception as e:
                    print(f"Error procesando blog {file}: {e}")

    print(f"Finalizado Blogs. Procesados: {contador}")

def procesar_soporte(base_path):
    """
    Módulo B: Soporte (PDFs + Extracción de Imágenes)
    Extrae texto de PDFs y guarda las imágenes incrustadas en disco.
    """
    print("--- Iniciando Módulo Soporte ---")
    ruta_soporte = os.path.join(base_path, "Support")
    contador = 0

    for root, dirs, files in os.walk(ruta_soporte):
        for file in files:
            if file.lower().endswith('.pdf'):
                ruta_pdf = os.path.join(root, file)
                try:
                    doc_pdf = fitz.open(ruta_pdf)
                    texto_completo = ""
                    imagenes_extraidas = []

                    # Iterar páginas
                    for num_pagina, pagina in enumerate(doc_pdf):
                        texto_completo += pagina.get_text() + " "

                        # Extraer imágenes de la página
                        lista_imagenes = pagina.get_images(full=True)
                        for img_index, img in enumerate(lista_imagenes):
                            xref = img[0]
                            base_imagen = doc_pdf.extract_image(xref)
                            bytes_imagen = base_imagen["image"]
                            ext_imagen = base_imagen["ext"]

                            # Guardar imagen en disco (Drive) para que VaLiK la vea luego
                            nombre_img = f"{file}_p{num_pagina}_i{img_index}.{ext_imagen}"
                            ruta_guardado = os.path.join(EXTRACTED_IMGS_PATH, nombre_img)

                            with open(ruta_guardado, "wb") as f_img:
                                f_img.write(bytes_imagen)

                            if es_imagen_valida(ruta_guardado):
                                imagenes_extraidas.append(ruta_guardado)
                            else:
                                os.remove(ruta_guardado) # Borrar si es basura (icono)

                    doc = {
                        "id": f"support_{file}_{contador}",
                        "fuente": "Support",
                        "ruta_origen": ruta_pdf,
                        "texto": texto_completo.strip(),
                        "imagenes": imagenes_extraidas,
                        "tipo": "multimodal" if imagenes_extraidas else "texto"
                    }
                    guardar_registro(doc, OUTPUT_FILE)
                    contador += 1

                except Exception as e:
                    print(f"Error procesando PDF {file}: {e}")

    print(f"Finalizado Soporte. Procesados: {contador}")

def procesar_noticias(base_path):
    """
    Módulo C: News (XML -> Texto)
    Asume estructura simple de XML.
    """
    print("--- Iniciando Módulo News ---")
    ruta_news = os.path.join(base_path, "News")
    contador = 0

    for root, dirs, files in os.walk(ruta_news):
        for file in files:
            if file.lower().endswith('.xml'):
                ruta_xml = os.path.join(root, file)
                try:
                    # Parsear XML
                    # Nota: Los XML a veces tienen formatos raros, usamos try-except robusto
                    with open(ruta_xml, 'r', encoding='utf-8', errors='ignore') as f:
                        contenido = f.read()

                    # Limpieza básica de XML mal formados si fuera necesario
                    root_xml = ET.fromstring(contenido)

                    # Intentar obtener todo el texto recursivamente
                    texto = "".join(root_xml.itertext()).strip()

                    if texto:
                        doc = {
                            "id": f"news_{file}_{contador}",
                            "fuente": "News",
                            "ruta_origen": ruta_xml,
                            "texto": texto,
                            "imagenes": [],
                            "tipo": "texto"
                        }
                        guardar_registro(doc, OUTPUT_FILE)
                        contador += 1

                except ET.ParseError:
                    # Fallback para XMLs corruptos: leer como texto plano
                    with open(ruta_xml, 'r', encoding='utf-8', errors='ignore') as f:
                        texto = f.read()
                    doc = {
                        "id": f"news_raw_{file}_{contador}",
                        "fuente": "News",
                        "ruta_origen": ruta_xml,
                        "texto": texto,
                        "imagenes": [],
                        "tipo": "texto"
                    }
                    guardar_registro(doc, OUTPUT_FILE)
                    contador += 1
                except Exception as e:
                    print(f"Error en News {file}: {e}")

    print(f"Finalizado News. Procesados: {contador}")

def procesar_databases(base_path):
    """
    Módulo D: Databases (CSV/XLS -> Texto Linealizado)
    Convierte filas de tablas en oraciones.
    """
    print("--- Iniciando Módulo Databases ---")
    ruta_db = os.path.join(base_path, "Databases")
    contador = 0

    if os.path.exists(ruta_db):
        for file in os.listdir(ruta_db):
            ruta_archivo = os.path.join(ruta_db, file)
            df = None

            try:
                if file.lower().endswith('.csv'):
                    df = pd.read_csv(ruta_archivo, encoding='latin-1', on_bad_lines='skip')
                elif file.lower().endswith(('.xls', '.xlsx')):
                    df = pd.read_excel(ruta_archivo)

                if df is not None:
                    # Linearización: Convertir cada fila en texto
                    # Ejemplo: "Columna1 es Valor1. Columna2 es Valor2."
                    columnas = df.columns
                    for index, row in df.iterrows():
                        oraciones = []
                        for col in columnas:
                            val = str(row[col]).strip()
                            if val and val.lower() != 'nan':
                                oraciones.append(f"{col}: {val}")

                        texto_fila = ". ".join(oraciones)

                        if texto_fila:
                            doc = {
                                "id": f"db_{file}_row{index}",
                                "fuente": "Databases",
                                "ruta_origen": ruta_archivo,
                                "texto": f"Registro de base de datos {file}. {texto_fila}",
                                "imagenes": [],
                                "tipo": "texto"
                            }
                            guardar_registro(doc, OUTPUT_FILE)
                            contador += 1
            except Exception as e:
                print(f"Error en Database {file}: {e}")

    print(f"Finalizado Databases. Procesados: {contador}")

def procesar_imagenes_sueltas(base_path):
    """
    Módulo E: Imágenes Sueltas (Solo Imagen)
    """
    print("--- Iniciando Módulo Imágenes Sueltas ---")
    ruta_imgs = os.path.join(base_path, "Images")
    contador = 0

    if os.path.exists(ruta_imgs):
        for file in os.listdir(ruta_imgs):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                ruta_img = os.path.join(ruta_imgs, file)

                if es_imagen_valida(ruta_img):
                    doc = {
                        "id": f"img_loose_{file}_{contador}",
                        "fuente": "Images",
                        "ruta_origen": ruta_img,
                        "texto": "", # Sin texto, el modelo VLM generará descripción luego
                        "imagenes": [ruta_img],
                        "tipo": "imagen"
                    }
                    guardar_registro(doc, OUTPUT_FILE)
                    contador += 1

    print(f"Finalizado Imágenes Sueltas. Procesados: {contador}")

## 4. EJECUCIÓN PRINCIPAL


In [ ]:
# 4. EJECUCIÓN PRINCIPAL
# ------------------------------------------------------
def ejecutar_etl():
    # Limpiar archivo de salida previo si se quiere reiniciar de cero
    if os.path.exists(OUTPUT_FILE):
        print("Aviso: El archivo de salida ya existe. Se agregarán datos nuevos (Append).")
        # Si prefieres borrarlo para empezar de cero, descomenta:
        # os.remove(OUTPUT_FILE)

    procesar_blogs(BASE_DATASET_PATH)
    procesar_soporte(BASE_DATASET_PATH)
    procesar_noticias(BASE_DATASET_PATH)
    procesar_databases(BASE_DATASET_PATH)
    procesar_imagenes_sueltas(BASE_DATASET_PATH)

    print(f"\n FASE 0 COMPLETADA.")
    print(f"Dataset unificado guardado en: {OUTPUT_FILE}")

# Ejecutar todo
if __name__ == "__main__":
    ejecutar_etl()

--- Iniciando Módulo Blogs ---
Finalizado Blogs. Procesados: 11
--- Iniciando Módulo Soporte ---
Finalizado Soporte. Procesados: 7
--- Iniciando Módulo News ---
Finalizado News. Procesados: 1455
--- Iniciando Módulo Databases ---
Finalizado Databases. Procesados: 2234
--- Iniciando Módulo Imágenes Sueltas ---
Finalizado Imágenes Sueltas. Procesados: 10

 FASE 0 COMPLETADA.
Dataset unificado guardado en: /content/drive/MyDrive/Proyecto_Tesis_Multimodal/data_procesada/dataset_procesado.jsonl


# **FASE 1: MOTOR VECTORIAL MACRO (ADAPTACIÓN CEMTM)**

## Dependencias

In [ ]:
# ==========================================
# FASE 1: MOTOR VECTORIAL MACRO (ADAPTACIÓN CEMTM)
# ==========================================

import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from tqdm.auto import tqdm
from google.colab import drive

## 1. CONFIGURACIÓN

In [ ]:
# 1. CONFIGURACIÓN
# ------------------------------------------------------
# Montar Drive si no está montado
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Rutas
BASE_PATH = "/content/drive/MyDrive/Proyecto_Tesis_Multimodal"
DATA_PATH = os.path.join(BASE_PATH, "data_procesada", "dataset_procesado.jsonl")
VECTORS_PATH = os.path.join(BASE_PATH, "vectores")
MODELS_PATH = os.path.join(BASE_PATH, "modelos")

os.makedirs(VECTORS_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

# Hiperparámetros
BATCH_SIZE = 32      # Tamaño del lote para no saturar RAM
NUM_TOPICS = 10      # Número de temas macro a descubrir (K)
EPOCHS = 20          # Épocas de entrenamiento para CEMTM
LEARNING_RATE = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")

Usando dispositivo: cuda


## 2. GENERACIÓN DE EMBEDDINGS (CLIP)


In [ ]:
# 2. GENERACIÓN DE EMBEDDINGS (CLIP) - CORREGIDO
# ------------------------------------------------------
def generar_embeddings():
    print("\n--- 2.1 Iniciando Generación de Embeddings con CLIP ---")

    # Si ya existen, borrar o saltar. Para asegurar que se arregle,
    # si el archivo anterior quedó corrupto o vacio, mejor lo sobrescribimos.
    if os.path.exists(os.path.join(VECTORS_PATH, "features.npy")):
        try:
            test_load = np.load(os.path.join(VECTORS_PATH, "features.npy"))
            if test_load.shape[0] > 0 and test_load.shape[2] == 512:
                print("Los vectores ya existen y parecen válidos. Saltando generación.")
                return
            else:
                print("Vectores con dimensiones incorrectas. Regenerando...")
        except:
            print("Archivo de vectores corrupto detectado. Regenerando...")

    # Cargar Modelo
    model_id = "openai/clip-vit-base-patch32"
    processor = CLIPProcessor.from_pretrained(model_id)
    model = CLIPModel.from_pretrained(model_id).to(DEVICE)
    model.eval()

    all_features = []
    all_targets = []
    doc_ids = []

    with open(DATA_PATH, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    print(f"Procesando {len(lines)} documentos...")

    for i in tqdm(range(0, len(lines), BATCH_SIZE)):
        batch_lines = lines[i : i + BATCH_SIZE]
        texts = []
        images = []

        # Preparar lote
        for line in batch_lines:
            doc = json.loads(line)

            # Texto
            txt = doc.get("texto", "")
            if len(txt) > 300: txt = txt[:300]
            if not txt: txt = "sin texto"

            # Imagen
            img_obj = None
            if doc.get("imagenes"):
                try:
                    img_path = doc["imagenes"][0]
                    if os.path.exists(img_path):
                        img_obj = Image.open(img_path).convert("RGB")
                except:
                    pass

            if img_obj is None:
                img_obj = Image.new("RGB", (224, 224), (0, 0, 0))

            texts.append(txt)
            images.append(img_obj)
            doc_ids.append(doc["id"])

        try:
            # Procesar inputs
            inputs = processor(
                text=texts,
                images=images,
                return_tensors="pt",
                padding='max_length',
                truncation=True,
                max_length=77
            ).to(DEVICE)

            with torch.no_grad():
                # Pasamos explícitamente solo los argumentos necesarios a cada sub-modelo

                # 1. Características de Texto
                text_outputs = model.text_model(
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask
                )
                # [Batch, 77, 512] - Ya está en 512, pero aplicamos proyección por coherencia
                text_feats = model.text_projection(text_outputs.last_hidden_state)

                # 2. Características de Imagen
                vision_outputs = model.vision_model(
                    pixel_values=inputs.pixel_values
                )
                vision_last = vision_outputs.last_hidden_state # [Batch, 50, 768] <-- AQUÍ ESTABA EL PROBLEMA

                # --- Proyectar de 768 a 512 ---
                vision_feats = model.visual_projection(vision_last) # [Batch, 50, 512] <-- AHORA SÍ ENCAJA

                # 3. Embedding Global (Target)
                # Recalculamos manualmente la proyección global para estar seguros
                # CLIPModel normaliza y proyecta internamente cuando se llama completo,
                # pero aquí lo reconstruimos o llamamos al modelo completo SOLO para el target.
                outputs_global = model(**inputs)
                global_embed = (outputs_global.text_embeds + outputs_global.image_embeds) / 2.0

            # Concatenamos para crear H (Multimodal Tokens) -> [Batch, 127, 512]
            H = torch.cat([text_feats, vision_feats], dim=1)

            all_features.append(H.cpu().numpy())
            all_targets.append(global_embed.cpu().numpy())

        except Exception as e:
            print(f"Error CRÍTICO en lote {i}: {e}")
            # Si falla un lote, continuamos, pero imprimimos el error real
            continue

    if len(all_features) == 0:
        print("Error: No se generaron vectores. Revisa los mensajes de error arriba.")
        return

    # Guardar
    print("Guardando vectores en disco...")
    try:
        all_features = np.concatenate(all_features, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)

        np.save(os.path.join(VECTORS_PATH, "features.npy"), all_features)
        np.save(os.path.join(VECTORS_PATH, "targets.npy"), all_targets)
        with open(os.path.join(VECTORS_PATH, "doc_ids.json"), 'w') as f:
            json.dump(doc_ids, f)

        print(f"Vectores generados exitosamente. Shape: {all_features.shape}")

    except ValueError as ve:
        print(f"Error de concatenación: {ve}")
        # Debugging: Imprimir formas si falla
        print("Formas de los primeros lotes:", [x.shape for x in all_features[:5]])
        print("Formas de los últimos lotes:", [x.shape for x in all_features[-5:]])

## 3. DEFINICIÓN DEL MODELO CEMTM (Red Neuronal)

In [ ]:
# 3. DEFINICIÓN DEL MODELO CEMTM (Red Neuronal)
# ------------------------------------------------------
class ImportanceNetwork(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2) # Salida: mu y log_sigma
        )

    def forward(self, h):
        # h: [Batch, Seq_Len, Dim]
        out = self.fc(h) # [Batch, Seq, 2]
        mu, log_sigma = out.chunk(2, dim=-1)
        sigma = torch.exp(log_sigma)

        # Reparametrization trick (Estocástico)
        epsilon = torch.randn_like(sigma)
        alpha = mu + epsilon * sigma # [Batch, Seq, 1]

        # Softmax sobre la dimensión de secuencia para obtener pesos beta
        beta = F.softmax(alpha, dim=1) # [Batch, Seq, 1]
        return beta, mu, sigma

class CEMTM(nn.Module):
    def __init__(self, input_dim, num_topics):
        super().__init__()
        self.input_dim = input_dim
        self.num_topics = num_topics

        # Módulos
        self.importance_net = ImportanceNetwork(input_dim)
        self.topic_encoder = nn.Linear(input_dim, num_topics) # W_t
        self.decoder = nn.Linear(num_topics, input_dim)       # W_d

    def forward(self, H):
        # H: [Batch, Seq, Dim]

        # 1. Calcular Importancia
        beta, mu, sigma = self.importance_net(H)

        # 2. Proyectar Tokens a Espacio de Tópicos (t_i)
        # T_tokens: [Batch, Seq, K]
        t_tokens = F.softmax(self.topic_encoder(H), dim=-1)

        # 3. Agregar ponderadamente para obtener Vector de Tópico del Documento (theta_d)
        # theta_d = sum(beta_i * t_i)
        # [Batch, 1, Seq] x [Batch, Seq, K] -> [Batch, 1, K]
        theta_d = torch.bmm(beta.transpose(1, 2), t_tokens).squeeze(1)

        # 4. Reconstrucción
        e_reconstructed = self.decoder(theta_d)

        return e_reconstructed, theta_d, mu, sigma, beta

## 4. ENTRENAMIENTO DE CEMTM

In [ ]:
# 4. ENTRENAMIENTO DE CEMTM
# ------------------------------------------------------
def entrenar_cemtm():
    if not os.path.exists(os.path.join(VECTORS_PATH, "features.npy")):
        print("No hay vectores para entrenar.")
        return

    # Cargar usando mmap_mode='r' para no saturar RAM si es muy grande
    try:
        features = np.load(os.path.join(VECTORS_PATH, "features.npy"), mmap_mode='r')
        targets = np.load(os.path.join(VECTORS_PATH, "targets.npy"), mmap_mode='r')
    except Exception as e:
        print(f"Error cargando vectores: {e}")
        return

    print("\n--- 2.2 Iniciando Entrenamiento de CEMTM ---")

    # Cargar datos
    features = np.load(os.path.join(VECTORS_PATH, "features.npy"))
    targets = np.load(os.path.join(VECTORS_PATH, "targets.npy"))

    # Convertir a Tensores
    dataset = TensorDataset(torch.tensor(features), torch.tensor(targets))
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

    # Inicializar Modelo
    model = CEMTM(input_dim=512, num_topics=NUM_TOPICS).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    model.train()
    print(f"Entrenando por {EPOCHS} épocas...")

    for epoch in range(EPOCHS):
        total_loss = 0
        for batch_H, batch_target in dataloader:
            batch_H = batch_H.to(DEVICE)
            batch_target = batch_target.to(DEVICE)

            optimizer.zero_grad()

            # Forward
            e_rec, theta_d, mu, sigma, beta = model(batch_H)

            # Loss Function (Según paper CEMTM: Reconstruction + KL + Entropy)
            loss_rec = F.mse_loss(e_rec, batch_target)

            # KL Divergence regularization (prior Normal(0,1))
            loss_kl = -0.5 * torch.sum(1 + torch.log(sigma**2) - mu**2 - sigma**2) / batch_H.size(0)

            # Entropy regularization (para hacer la atención dispersa/enfocada)
            # Minimizar entropía negativa = Maximizar certeza
            loss_ent = -torch.sum(beta * torch.log(beta + 1e-9)) / batch_H.size(0)

            # Total Loss (Pesos lambda simplificados)
            loss = loss_rec + 0.1 * loss_kl + 0.01 * loss_ent

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if (epoch+1) % 5 == 0:
            print(f"Época {epoch+1}/{EPOCHS} - Loss: {total_loss/len(dataloader):.4f}")

    # Guardar modelo entrenado
    torch.save(model.state_dict(), os.path.join(MODELS_PATH, "cemtm_model.pth"))
    print("Modelo CEMTM entrenado y guardado.")
    return model

## 5. INFERENCIA Y ASIGNACIÓN DE TEMAS

In [ ]:
# 5. INFERENCIA Y ASIGNACIÓN DE TEMAS
# ------------------------------------------------------
def inferencia_temas():
    print("\n--- 2.3 Asignando Temas a Documentos ---")

    if not os.path.exists(os.path.join(MODELS_PATH, "cemtm_model.pth")):
        return

    # Cargar
    features = np.load(os.path.join(VECTORS_PATH, "features.npy"))
    with open(os.path.join(VECTORS_PATH, "doc_ids.json"), 'r') as f:
        doc_ids = json.load(f)

    model = CEMTM(input_dim=512, num_topics=NUM_TOPICS).to(DEVICE)
    model.load_state_dict(torch.load(os.path.join(MODELS_PATH, "cemtm_model.pth")))
    model.eval()

    dataset = TensorDataset(torch.tensor(features))
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

    all_topics = []

    with torch.no_grad():
        for batch_H in dataloader:
            batch_H = batch_H[0].to(DEVICE)
            _, theta_d, _, _, _ = model(batch_H)
            # theta_d es [Batch, K] (distribución de temas)
            # Tomamos el tema con mayor peso
            top_topic = torch.argmax(theta_d, dim=1).cpu().numpy()
            all_topics.extend(top_topic)

    # Guardar Resultados Finales de Fase 1
    resultados = []
    for doc_id, topic in zip(doc_ids, all_topics):
        resultados.append({"id": doc_id, "topic_id": int(topic)})

    with open(os.path.join(BASE_PATH, "metadata_con_temas_fase1.json"), 'w') as f:
        json.dump(resultados, f)

    print(f"Asignación completada. Guardado en 'metadata_con_temas_fase1.json'")

## EJECUTAR TODO


In [ ]:
# EJECUTAR TODO
if __name__ == "__main__":
    generar_embeddings()
    entrenar_cemtm()
    inferencia_temas()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



--- 2.1 Iniciando Generación de Embeddings con CLIP ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Procesando 3717 documentos...


  0%|          | 0/117 [00:00<?, ?it/s]

Guardando vectores en disco...
Vectores generados exitosamente. Shape: (3717, 127, 512)

--- 2.2 Iniciando Entrenamiento de CEMTM ---
Entrenando por 20 épocas...
Época 5/20 - Loss: 0.0444
Época 10/20 - Loss: 0.0439
Época 15/20 - Loss: 0.0438
Época 20/20 - Loss: 0.0437
Modelo CEMTM entrenado y guardado.

--- 2.3 Asignando Temas a Documentos ---
Asignación completada. Guardado en 'metadata_con_temas_fase1.json'


# **FASE 2: LÓGICA DE CONTROL (MIDDLEWARE)**

## Dependencias

In [ ]:
import os
import json
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from google.colab import drive

## 1. CONFIGURACIÓN


In [ ]:
# 1. CONFIGURACIÓN
# ------------------------------------------------------
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/Proyecto_Tesis_Multimodal"
VECTORS_PATH = os.path.join(BASE_PATH, "vectores")
# Usaremos este archivo para leer IDs, pero sobrescribiremos la asignación de temas
METADATA_FILE = os.path.join(BASE_PATH, "metadata_con_temas_fase1.json")
OUTPUT_STRUCTURE = os.path.join(BASE_PATH, "graph_structure.json")

# --- REGLAS DE NEGOCIO AJUSTADAS ---
# Vamos a dividir en 20 grupos. 3717 / 20 = ~185 docs por grupo.
# Subimos el umbral para permitir que estos grupos pasen a VaLiK.
UMBRAL_CANTIDAD_DOCS = 250
UMBRAL_VARIANZA = 0.20

## 2. CARGA DE DATOS


In [ ]:
# 2. CARGA DE DATOS
# ------------------------------------------------------
print("--- Cargando vectores existentes ---")

try:
    embeddings = np.load(os.path.join(VECTORS_PATH, "targets.npy"))
    with open(os.path.join(VECTORS_PATH, "doc_ids.json"), 'r') as f:
        doc_ids_list = json.load(f)

    # Cargamos la metadata original solo para ver qué pasó, pero no la usaremos para agrupar
    with open(METADATA_FILE, 'r') as f:
        old_metadata = json.load(f)

    print(f"Datos cargados. Total documentos: {len(embeddings)}")

except Exception as e:
    print(f"Error crítico cargando archivos: {e}")
    exit()

--- Cargando vectores existentes ---
Datos cargados. Total documentos: 3717


## 3. REPARACIÓN: FORZAR CLUSTERING (K-MEANS)


In [ ]:
# 3. REPARACIÓN: FORZAR CLUSTERING (K-MEANS)
# ------------------------------------------------------
print("\n--- Ejecutando Re-Clustering Forzado (K=20) ---")
print("Dividiendo el nodo gigante en sub-temas manejables...")

# Forzamos 20 clusters para tener grupos de tamaño manejable (~185 docs)
kmeans = KMeans(n_clusters=20, random_state=42, n_init=10)
nuevos_labels = kmeans.fit_predict(embeddings)

# Reconstruimos el diccionario de temas con los nuevos labels
topics_dict = {}
for idx, label in enumerate(nuevos_labels):
    label = int(label)
    doc_id = doc_ids_list[idx]

    if label not in topics_dict:
        topics_dict[label] = []
    topics_dict[label].append(doc_id)

print(f"Re-clustering completado. Ahora tenemos {len(topics_dict)} temas distintos.")

# Crear mapeo rápido ID -> Indice
id_to_index = {doc_id: i for i, doc_id in enumerate(doc_ids_list)}


--- Ejecutando Re-Clustering Forzado (K=20) ---
Dividiendo el nodo gigante en sub-temas manejables...
Re-clustering completado. Ahora tenemos 20 temas distintos.


## 4. CÁLCULO DE MÉTRICAS Y SEMÁFORO (LOGICA MIDDLEWARE)


In [ ]:
# 4. CÁLCULO DE MÉTRICAS Y SEMÁFORO (LOGICA MIDDLEWARE)
# ------------------------------------------------------
print("\n--- Analizando calidad de los nuevos temas ---")

topic_stats = {}
nodos_terminales = 0
nodos_grouper = 0

for topic_id, docs_in_topic in topics_dict.items():
    # Obtener índices
    indices = [id_to_index[did] for did in docs_in_topic]

    # Extraer vectores
    vectors_topic = embeddings[indices]

    # Métricas
    centroid = np.mean(vectors_topic, axis=0, keepdims=True)
    distances = cdist(vectors_topic, centroid, metric='cosine')
    semantic_variance = np.mean(distances)
    count = len(docs_in_topic)

    # --- REGLA DEL SEMÁFORO ---
    es_grande = count > UMBRAL_CANTIDAD_DOCS
    es_difuso = semantic_variance > UMBRAL_VARIANZA

    if es_grande or es_difuso:
        node_type = "grouper"
        nodos_grouper += 1
        accion = "Zoom Requerido"
    else:
        node_type = "terminal"
        nodos_terminales += 1
        accion = "Activar VaLiK"

    topic_stats[topic_id] = {
        "topic_id": topic_id,
        "type": node_type,
        "doc_count": count,
        "semantic_variance": float(semantic_variance),
        "documents": docs_in_topic, # Lista de IDs
        "reason": f"Docs: {count} vs {UMBRAL_CANTIDAD_DOCS}, Var: {semantic_variance:.3f}"
    }

    print(f"Tema {topic_id}: {count} docs | Var: {semantic_variance:.3f} | -> {node_type.upper()}")


--- Analizando calidad de los nuevos temas ---
Tema 3: 207 docs | Var: 0.066 | -> TERMINAL
Tema 16: 10 docs | Var: 0.099 | -> TERMINAL
Tema 7: 147 docs | Var: 0.065 | -> TERMINAL
Tema 17: 161 docs | Var: 0.054 | -> TERMINAL
Tema 12: 85 docs | Var: 0.114 | -> TERMINAL
Tema 8: 105 docs | Var: 0.071 | -> TERMINAL
Tema 1: 224 docs | Var: 0.057 | -> TERMINAL
Tema 9: 96 docs | Var: 0.079 | -> TERMINAL
Tema 18: 186 docs | Var: 0.063 | -> TERMINAL
Tema 10: 129 docs | Var: 0.084 | -> TERMINAL
Tema 5: 123 docs | Var: 0.050 | -> TERMINAL
Tema 15: 65 docs | Var: 0.028 | -> TERMINAL
Tema 2: 587 docs | Var: 0.004 | -> GROUPER
Tema 0: 518 docs | Var: 0.005 | -> GROUPER
Tema 19: 328 docs | Var: 0.001 | -> GROUPER
Tema 11: 206 docs | Var: 0.001 | -> TERMINAL
Tema 6: 172 docs | Var: 0.001 | -> TERMINAL
Tema 13: 243 docs | Var: 0.001 | -> TERMINAL
Tema 4: 115 docs | Var: 0.022 | -> TERMINAL
Tema 14: 10 docs | Var: 0.048 | -> TERMINAL


## 5. GUARDAR ESTRUCTURA

In [ ]:
# 5. GUARDAR ESTRUCTURA
# ------------------------------------------------------
graph_structure = {
    "global_stats": {
        "total_docs": len(embeddings),
        "total_topics": len(topics_dict),
        "terminal_nodes": nodos_terminales,
        "grouper_nodes": nodos_grouper
    },
    "nodes": topic_stats
}

with open(OUTPUT_STRUCTURE, 'w') as f:
    json.dump(graph_structure, f, indent=4)

print("\n========================================")
print(f"FASE 2 (REPARADA) COMPLETADA")
print(f"Estructura guardada en: {OUTPUT_STRUCTURE}")
print(f"Resultado: {nodos_terminales} nodos TERMINALES listos para VaLiK.")
print("========================================")


FASE 2 (REPARADA) COMPLETADA
Estructura guardada en: /content/drive/MyDrive/Proyecto_Tesis_Multimodal/graph_structure.json
Resultado: 17 nodos TERMINALES listos para VaLiK.


# **Fase 3: Generación de Conocimiento Micro (Adaptación VaLiK)**

Aplicao una estrategia de "Carga y Descarga Secuencial":

    Cargamos Modelo de Visión → Procesamos imágenes → Borrar Modelo de la GPU.

    Cargamos Modelo CLIP → Verificamos descripciones → Borrar Modelo de la GPU.

    Cargamos Modelo LLM → Extraemos Grafo → Borrar Modelo de la GPU.

# **FASE 3: GENERACIÓN DE CONOCIMIENTO MICRO (VaLiK)**

## Dependencias Previas

In [ ]:
# 1. INSTALACIÓN DE DEPENDENCIAS
# ------------------------------------------------------
# Ejecuta esto una sola vez al inicio de la sesión
import os
import subprocess

# Función para instalar silenciosamente
def install_dependencies():
    print("Instalando dependencias necesarias...")
    subprocess.run(["pip", "uninstall", "-y", "sentence-transformers"], check=False)
    subprocess.run(["pip", "install", "-q", "git+https://github.com/huggingface/transformers"], check=True)
    subprocess.run(["pip", "install", "-q", "accelerate", "bitsandbytes", "qwen-vl-utils"], check=True)
    print("Dependencias instaladas.")

try:
    import transformers
    import qwen_vl_utils
except ImportError:
    install_dependencies()

## Dependencias

In [ ]:
import json
import gc
import torch
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
from google.colab import drive

# Importaciones de librerías
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import CLIPProcessor, CLIPModel, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

## 1. CONFIGURACIÓN


In [ ]:
# 2. CONFIGURACIÓN
# ------------------------------------------------------
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/Proyecto_Tesis_Multimodal"
STRUCTURE_FILE = os.path.join(BASE_PATH, "graph_structure.json")
# Nombre diferenciado para el resultado optimizado
KNOWLEDGE_OUTPUT = os.path.join(BASE_PATH, "final_knowledge_graph_optimizado.json")
DATA_PATH = os.path.join(BASE_PATH, "data_procesada", "dataset_procesado.jsonl")

# Archivos intermedios
DESC_FILE = os.path.join(BASE_PATH, "fase3_descripciones_opt.json")
VERIFIED_FILE = os.path.join(BASE_PATH, "fase3_descripciones_verificadas_opt.json")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# REGLA DE OPTIMIZACIÓN: Solo procesar los N primeros documentos de cada tema
DOCS_PER_TOPIC_LIMIT = 15

print(f"Usando dispositivo: {DEVICE}")
print(f"Límite de documentos por tema: {DOCS_PER_TOPIC_LIMIT}")

Usando dispositivo: cuda
Límite de documentos por tema: 15


## 2. PREPARACIÓN DE DATOS


In [ ]:
# 3. PREPARACIÓN DE DATOS (CON MUESTREO)
# ------------------------------------------------------
def obtener_documentos_filtrados():
    print("--- Seleccionando documentos representativos por tema ---")

    if not os.path.exists(STRUCTURE_FILE):
        print("Error: No se encuentra graph_structure.json. Ejecuta la Fase 2 primero.")
        return []

    with open(STRUCTURE_FILE, 'r') as f:
        structure = json.load(f)

    selected_ids = set()
    stats_count = 0

    # Iterar por cada tema terminal
    for topic_id, data in structure["nodes"].items():
        if data["type"] == "terminal":
            docs_in_topic = data["documents"]
            # TOMAMOS SOLO LOS PRIMEROS N DOCUMENTOS
            seleccionados = docs_in_topic[:DOCS_PER_TOPIC_LIMIT]
            selected_ids.update(seleccionados)
            stats_count += len(seleccionados)

    print(f"Total documentos seleccionados para procesamiento profundo: {stats_count}")

    # Cargar contenido real
    docs_to_process = []
    with open(DATA_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            doc = json.loads(line)
            if doc["id"] in selected_ids:
                docs_to_process.append(doc)

    return docs_to_process

## 3. PASO 1: VISIÓN A LENGUAJE (Qwen2-VL)


In [ ]:
# 4. PASO 3.1: VISIÓN A LENGUAJE (Qwen2-VL)
# ------------------------------------------------------
def generar_descripciones(docs):
    print("\n--- PASO 3.1: Generando Descripciones de Imágenes (Qwen2-VL) ---")

    if os.path.exists(DESC_FILE):
        print("Archivo de descripciones encontrado. Saltando generación.")
        with open(DESC_FILE, 'r') as f:
            return json.load(f)

    # Filtrar solo docs con imágenes válidas
    docs_con_imagen = []
    for d in docs:
        if d.get("imagenes") and len(d["imagenes"]) > 0:
            if os.path.exists(d["imagenes"][0]):
                docs_con_imagen.append(d)

    print(f"Procesando {len(docs_con_imagen)} imágenes...")

    if len(docs_con_imagen) == 0:
        return {d["id"]: "" for d in docs}

    # Configuración 4-bit
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    print("Cargando Qwen2-VL-2B-Instruct...")
    try:
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            "Qwen/Qwen2-VL-2B-Instruct",
            quantization_config=bnb_config,
            device_map="auto"
        )
        processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
    except Exception as e:
        print(f"Error cargando modelo de visión: {e}")
        return {}

    descripciones = {}

    for doc in tqdm(docs_con_imagen):
        try:
            img_path = doc["imagenes"][0]

            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": img_path},
                        {"type": "text", "text": "Describe esta imagen detalladamente enumerando objetos y acciones visibles."}
                    ],
                }
            ]

            text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)

            inputs = processor(
                text=[text_input],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            ).to(DEVICE)

            generated_ids = model.generate(**inputs, max_new_tokens=100)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]

            descripciones[doc["id"]] = output_text

        except Exception as e:
            # Captura silenciosa de errores de memoria en imágenes gigantes
            descripciones[doc["id"]] = ""

    # Guardar
    with open(DESC_FILE, 'w') as f:
        json.dump(descripciones, f)

    # Limpieza de memoria
    del model
    del processor
    try:
        del inputs
        del generated_ids
    except: pass
    torch.cuda.empty_cache()
    gc.collect()
    print("Descripciones generadas y memoria liberada.")
    return descripciones

## 4. PASO 2: VERIFICACIÓN (CLIP)

In [ ]:
# 5. PASO 3.2: VERIFICACIÓN (CLIP)
# ------------------------------------------------------
def verificar_descripciones(docs, descripciones_dict):
    print("\n--- PASO 3.2: Verificación de Similitud (CLIP) ---")

    if os.path.exists(VERIFIED_FILE):
        print("Archivo verificado encontrado. Saltando verificación.")
        with open(VERIFIED_FILE, 'r') as f:
            return json.load(f)

    model_id = "openai/clip-vit-base-patch32"
    model = CLIPModel.from_pretrained(model_id).to(DEVICE)
    processor = CLIPProcessor.from_pretrained(model_id)

    descripciones_finales = {}
    UMBRAL_CLIP = 0.18 # Umbral ligeramente permisivo para descripciones complejas

    for doc in tqdm(docs):
        doc_id = doc["id"]
        texto_generado = descripciones_dict.get(doc_id, "")

        if not texto_generado or not doc.get("imagenes"):
            descripciones_finales[doc_id] = ""
            continue

        try:
            img_path = doc["imagenes"][0]
            if not os.path.exists(img_path):
                 descripciones_finales[doc_id] = ""
                 continue

            image = Image.open(img_path).convert("RGB")
            texto_clip = texto_generado[:77]

            inputs = processor(text=[texto_clip], images=image, return_tensors="pt", padding=True).to(DEVICE)

            with torch.no_grad():
                outputs = model(**inputs)
                logits_per_image = outputs.logits_per_image
                score = logits_per_image.item() / 100.0

            if score > UMBRAL_CLIP:
                descripciones_finales[doc_id] = texto_generado
            else:
                descripciones_finales[doc_id] = ""

        except Exception as e:
            descripciones_finales[doc_id] = ""

    with open(VERIFIED_FILE, 'w') as f:
        json.dump(descripciones_finales, f)

    del model
    del processor
    torch.cuda.empty_cache()
    gc.collect()
    print("Verificación completada y memoria liberada.")
    return descripciones_finales

## 5. PASO 3: TEXTO A GRAFO (LightRAG con Qwen2.5)

In [ ]:
# 6. PASO 3.3: TEXTO A GRAFO (Qwen2.5)
# ------------------------------------------------------
def extraer_conocimiento(docs, descripciones_verificadas):
    print("\n--- PASO 3.3: Extracción de Entidades (Qwen2.5) ---")

    if os.path.exists(KNOWLEDGE_OUTPUT):
        print("Grafo final ya existe. Proceso terminado.")
        return

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    print("Cargando Qwen2.5-1.5B-Instruct...")
    model_id = "Qwen/Qwen2.5-1.5B-Instruct"
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto"
        )
    except Exception as e:
         print(f"Error cargando LLM: {e}")
         return

    grafo_final = []

    # Prompt optimizado para velocidad y estructura
    prompt_template = """Extrae entidades y relaciones del siguiente texto e imagen.
    Formato: Sujeto | Relación | Objeto

    Contexto Visual: {visual}
    Texto: {text}

    Tripletas:"""

    for doc in tqdm(docs):
        doc_id = doc["id"]
        texto_original = doc.get("texto", "")
        texto_visual = descripciones_verificadas.get(doc_id, "")

        # Recorte estratégico: 800 chars de texto + 200 de visual
        contexto_text = texto_original[:800]
        contexto_vis = texto_visual[:200]

        input_text = prompt_template.format(text=contexto_text, visual=contexto_vis)

        messages = [{"role": "user", "content": input_text}]

        try:
            text_input = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            model_inputs = tokenizer([text_input], return_tensors="pt").to(DEVICE)

            # Generación rápida (max 100 tokens)
            generated_ids = model.generate(**model_inputs, max_new_tokens=100, temperature=0.1)
            generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

            response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

            tripletas = []
            for line in response.split('\n'):
                if '|' in line and len(line.split('|')) >= 3:
                    parts = line.split('|')
                    tripletas.append({
                        "head": parts[0].strip(),
                        "relation": parts[1].strip(),
                        "tail": parts[2].strip()
                    })

            grafo_final.append({
                "doc_id": doc_id,
                "triplets": tripletas
            })

        except Exception as e:
            continue

    # Guardar Resultado Final
    with open(KNOWLEDGE_OUTPUT, 'w') as f:
        json.dump(grafo_final, f, indent=4)

    print(f"Grafo optimizado guardado en {KNOWLEDGE_OUTPUT}")

## 7. EJECUCIÓN

In [ ]:
# 7. EJECUCIÓN
# ------------------------------------------------------
def ejecutar_fase3():
    # 1. Obtener datos (Subset Filtrado)
    docs = obtener_documentos_filtrados()

    if not docs:
        print("No hay documentos seleccionados.")
        return

    # 2. Pipeline
    descripciones = generar_descripciones(docs)
    descripciones_verificadas = verificar_descripciones(docs, descripciones)
    extraer_conocimiento(docs, descripciones_verificadas)

    print("\n FASE 3 OPTIMIZADA COMPLETADA EXITOSAMENTE")

if __name__ == "__main__":
    ejecutar_fase3()

--- Seleccionando documentos representativos por tema ---
Total documentos seleccionados para procesamiento profundo: 245

--- PASO 3.1: Generando Descripciones de Imágenes (Qwen2-VL) ---
Procesando 24 imágenes...
Cargando Qwen2-VL-2B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_section'}


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


...

  0%|          | 0/245 [00:00<?, ?it/s]

Verificación completada y memoria liberada.

--- PASO 3.3: Extracción de Entidades (Qwen2.5) ---
Cargando Qwen2.5-1.5B-Instruct...


Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

  0%|          | 0/245 [00:00<?, ?it/s]

Grafo optimizado guardado en /content/drive/MyDrive/Proyecto_Tesis_Multimodal/final_knowledge_graph_optimizado.json

 FASE 3 OPTIMIZADA COMPLETADA EXITOSAMENTE


# **FASE 4: VISUALIZACIÓN Y PERSISTENCIA FINAL**

## Dependencias

In [1]:
# ==========================================
# FASE 4: VISUALIZACIÓN Y PERSISTENCIA FINAL
# ==========================================

!pip install -q deep-translator

import os
import json
import string
import networkx as nx
import nltk
from collections import Counter
from nltk.corpus import stopwords
from google.colab import drive
from deep_translator import GoogleTranslator # ### MODIFICACIÓN: Importar Traductor

## 1. CONFIGURACIÓN


In [2]:
# 1. CONFIGURACIÓN E INSTALACIÓN DE UTILIDADES
# ------------------------------------------------------
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.update(['image', 'jpg', 'gif', 'page', 'date', 'posted', 'blog', 'http', 'com', 'www', 'html'])

BASE_PATH = "/content/drive/MyDrive/Proyecto_Tesis_Multimodal"

# Entradas
STRUCTURE_FILE = os.path.join(BASE_PATH, "graph_structure.json")
KNOWLEDGE_FILE = os.path.join(BASE_PATH, "final_knowledge_graph_optimizado.json")
DATA_PATH = os.path.join(BASE_PATH, "data_procesada", "dataset_procesado.jsonl")

# Salidas
OUTPUT_DIR = os.path.join(BASE_PATH, "grafo_final")
os.makedirs(OUTPUT_DIR, exist_ok=True)

HTML_OUTPUT = os.path.join(OUTPUT_DIR, "visor_grafo_espanol.html")

# ### MODIFICACIÓN: Inicializar Traductor y Caché (para no traducir lo mismo 2 veces)
translator = GoogleTranslator(source='auto', target='es')
translation_cache = {}

def traducir(texto):
    """Traduce texto al español de forma segura."""
    # Validación inicial agresiva
    if not texto: return ""
    texto = str(texto).strip().replace('_', ' ')
    if not texto: return ""

    if texto in translation_cache:
        return translation_cache[texto]

    try:
        # Traducimos
        if len(texto) > 50:
            translated = translator.translate(texto[:50]) + "..."
        else:
            translated = translator.translate(texto)

        # SIEMPRE verificamos que la respuesta no sea None
        if translated is None:
            return texto

        translation_cache[texto] = translated
        return translated
    except:
        # Si algo falla (internet, error de librería), devolvemos el original
        return texto

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2. FUNCIONES DE LIMPIEZA Y ETIQUETADO


In [3]:
# 2. FUNCIONES DE LIMPIEZA
# ------------------------------------------------------
def limpiar_texto(texto):
    if not texto: return []
    texto = texto.lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    tokens = [w for w in texto.split() if w not in stop_words and len(w) > 3]
    return tokens

def generar_nombres_temas(structure, all_docs_content):
    print("Generando nombres semánticos (y traduciendo)...")
    topic_labels = {}

    for topic_id, data in structure["nodes"].items():
        doc_ids = data["documents"]
        texto_acumulado = []
        for did in doc_ids:
            txt = all_docs_content.get(did, {}).get("texto_completo", "")
            texto_acumulado.extend(limpiar_texto(txt))

        if texto_acumulado:
            common_words = Counter(texto_acumulado).most_common(4)
            # Traducimos las palabras clave
            palabras_es = [traducir(w[0]).capitalize() for w in common_words]
            label = ", ".join(palabras_es)
        else:
            label = "Sin contenido textual"

        topic_labels[topic_id] = f"Tema {topic_id}: {label}"

    return topic_labels

## 3. CARGA DE DATOS


In [5]:
# 3. CARGA DE DATOS
# ------------------------------------------------------
def cargar_datos_completos():
    print("Cargando datos...")
    with open(STRUCTURE_FILE, 'r') as f:
        structure = json.load(f)

    kg_dict = {}
    if os.path.exists(KNOWLEDGE_FILE):
        with open(KNOWLEDGE_FILE, 'r') as f:
            kg_data = json.load(f)
            for item in kg_data:
                kg_dict[item["doc_id"]] = item.get("triplets", [])

    all_docs_content = {}
    with open(DATA_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            doc = json.loads(line)
            all_docs_content[doc["id"]] = {
                "texto_completo": doc.get("texto", ""),
                "tipo": doc.get("tipo", "texto"),
                "fuente": doc.get("fuente", "Desconocida")
            }

    return structure, kg_dict, all_docs_content

## 3. CONSTRUCCIÓN DEL GRAFO (NetworkX)


In [6]:
# 4. CONSTRUCCIÓN DEL GRAFO (NetworkX)
# ------------------------------------------------------
def construir_grafo_semantico():
    structure, kg_dict, all_docs_content = cargar_datos_completos()
    topic_names = generar_nombres_temas(structure, all_docs_content)

    G = nx.DiGraph()
    print("Construyendo nodos y traduciendo relaciones...")

    # --- NIVEL 1: TEMAS ---
    for topic_id, data in structure["nodes"].items():
        node_id = f"TOPIC_{topic_id}"
        tipo_nodo = data["type"]
        count = data["doc_count"]

        semantic_label = topic_names.get(topic_id, f"Tema {topic_id}")
        full_label = f"{semantic_label}\n({count} docs)"

        color = "#FF9900" if tipo_nodo == "grouper" else "#3366CC"

        # ### MODIFICACIÓN: Eliminada la "Varianza" del title
        G.add_node(node_id, label=full_label, title=f"Grupo de {count} documentos",
                   color=color, size=45, group="tema",
                   font={'size': 20, 'face': 'arial', 'color': 'white'})

        # --- NIVEL 2: DOCUMENTOS ---
        if tipo_nodo == "terminal":
            docs_ids = data["documents"]

            for doc_id in docs_ids:
                if doc_id in kg_dict:
                    doc_node_id = f"DOC_{doc_id}"
                    info = all_docs_content.get(doc_id, {})

                    texto_raw = info["texto_completo"]
                    # Snippet para el label
                    snippet = (texto_raw[:30] + '...') if len(texto_raw) > 30 else texto_raw
                    # Traducimos snippet si es corto, o lo dejamos raw si es nombre

                    tipo_doc = info["tipo"]
                    doc_color = "#109618" if tipo_doc == "multimodal" else "#990099"

                    # Tooltip (Hover)
                    tooltip_text = texto_raw[:300] + "..."

                    # ### MODIFICACIÓN: Propiedad 'hidden' para el JS
                    G.add_node(doc_node_id, label=snippet, title=tooltip_text,
                               color=doc_color, size=20, group="documento")

                    G.add_edge(node_id, doc_node_id, color="#CCCCCC", width=1)

                    # --- NIVEL 3: ENTIDADES ---
                    triplets = kg_dict[doc_id]

                    for triplet in triplets[:8]:
                        # Usamos .get() para evitar errores si falta alguna clave
                        head = triplet.get("head", "Entidad")
                        relation = triplet.get("relation", "relacionado con")
                        tail = triplet.get("tail", "Entidad")

                        # ### MODIFICACIÓN: Traducción con Red de Seguridad
                        # Si traducir() falla, usamos el original. Si el original es None, usamos cadena vacía.
                        tx_head = traducir(head)
                        head_es = (tx_head if tx_head else head).title()

                        tx_tail = traducir(tail)
                        tail_es = (tx_tail if tx_tail else tail).title()

                        tx_rel = traducir(relation)
                        relation_es = (tx_rel if tx_rel else relation).lower()

                        # IDs internos (basados en el original en inglés para evitar duplicados por traducción)
                        head_id = f"ENT_{str(head).replace(' ', '_').lower()}"
                        tail_id = f"ENT_{str(tail).replace(' ', '_').lower()}"

                        if not G.has_node(head_id):
                            G.add_node(head_id, label=head_es, color="#E0E0E0", size=10, group="entidad", font={'size': 10, 'color': 'black'})
                        if not G.has_node(tail_id):
                            G.add_node(tail_id, label=tail_es, color="#E0E0E0", size=10, group="entidad", font={'size': 10, 'color': 'black'})

                        G.add_edge(doc_node_id, head_id, color="#AAAAAA", dashes=True)
                        G.add_edge(head_id, tail_id, label=relation_es, font={'size': 9, 'align': 'middle'}, color="#555555", arrows='to')

    print(f"Grafo construido. Nodos: {G.number_of_nodes()}")
    return G

## 4. EXPORTACIÓN A HTML (Vis.js)


In [7]:
# 5. GENERAR HTML (Vis.js con Lógica de Expansión)
# ------------------------------------------------------
def generar_html(G):
    print("Generando HTML interactivo...")
    nodes = []
    edges = []

    for node_id, attr in G.nodes(data=True):
        # ### MODIFICACIÓN: Ocultar inicialmente si no es un TEMA
        es_tema = (attr.get("group") == "tema")

        nodes.append({
            "id": node_id,
            "label": attr.get("label", ""),
            "title": attr.get("title", ""),
            "color": attr.get("color", ""),
            "size": attr.get("size", 10),
            "font": attr.get("font", {}),
            "group": attr.get("group", ""),
            "hidden": not es_tema # TRUE para Docs/Entidades, FALSE para Temas
        })

    for u, v, attr in G.edges(data=True):
        edge_data = {"from": u, "to": v}
        if "label" in attr:
            edge_data["label"] = attr["label"]
            edge_data["font"] = attr.get("font", {})
        if "dashes" in attr: edge_data["dashes"] = True
        if "color" in attr: edge_data["color"] = {"color": attr["color"]}

        # Aristas también ocultas si conectan cosas ocultas
        # La lógica de vis.js oculta aristas automáticamente si los nodos están ocultos,
        # pero podemos forzarlo.
        edge_data["hidden"] = True
        edges.append(edge_data)

    data_json = json.dumps({"nodes": nodes, "edges": edges})

    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Grafo Multimodal (Español)</title>
        <script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
        <style type="text/css">
            body {{ margin: 0; font-family: 'Segoe UI', sans-serif; overflow: hidden; }}
            #mynetwork {{ width: 100vw; height: 100vh; background-color: #ffffff; }}
            .legend {{
                position: absolute; top: 10px; left: 10px;
                background: rgba(255, 255, 255, 0.95);
                padding: 15px; border-radius: 8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.2);
                z-index: 100; max-width: 250px;
            }}
            .dot {{ height: 12px; width: 12px; display: inline-block; border-radius: 50%; margin-right: 8px; }}
            .instr {{ font-size: 0.85em; color: #555; margin-top: 10px; border-top: 1px solid #ccc; padding-top: 5px;}}
        </style>
    </head>
    <body>
        <div class="legend">
            <h3 style="margin-top:0;">Grafo de Conocimiento</h3>
            <div><span class="dot" style="background-color: #3366CC;"></span><b>TEMA</b> (Click para expandir)</div>
            <div><span class="dot" style="background-color: #109618;"></span><b>DOC:</b> Multimodal</div>
            <div><span class="dot" style="background-color: #990099;"></span><b>DOC:</b> Texto</div>
            <div><span class="dot" style="background-color: #E0E0E0;"></span><b>ENTIDAD</b></div>
            <div class="instr">
                <b>Instrucciones:</b><br>
                1. Al inicio solo ves los TEMAS.<br>
                2. Haz <b>DOBLE CLICK</b> en un Tema azul para ver sus documentos.<br>
                3. Haz click en un Documento para ver sus entidades.
            </div>
        </div>
        <div id="mynetwork"></div>

        <script type="text/javascript">
            var rawData = {data_json};

            // Convertir a DataSet de Vis.js para manipulación dinámica
            var nodes = new vis.DataSet(rawData.nodes);
            var edges = new vis.DataSet(rawData.edges);
            var data = {{ nodes: nodes, edges: edges }};

            var container = document.getElementById('mynetwork');
            var options = {{
                nodes: {{ shape: 'dot' }},
                physics: {{
                    stabilization: false,
                    barnesHut: {{ gravitationalConstant: -30000, springConstant: 0.04, avoidOverlap: 0.5 }}
                }},
                interaction: {{ hover: true, navigationButtons: true }}
            }};

            var network = new vis.Network(container, data, options);

            // ### MODIFICACIÓN: Lógica de Click para Expandir
            network.on("click", function (params) {{
                if (params.nodes.length > 0) {{
                    var nodeId = params.nodes[0];
                    var clickedNode = nodes.get(nodeId);

                    // Solo expandir si es Tema o Documento
                    if (clickedNode.group === 'tema' || clickedNode.group === 'documento') {{
                        // Obtener nodos conectados
                        var connectedNodes = network.getConnectedNodes(nodeId);
                        var connectedEdges = network.getConnectedEdges(nodeId);

                        // Actualizar nodos conectados a hidden: false
                        var nodesToUpdate = [];
                        connectedNodes.forEach(function(id) {{
                            nodesToUpdate.push({{id: id, hidden: false}});
                        }});
                        nodes.update(nodesToUpdate);

                        // Actualizar aristas conectadas a hidden: false
                        var edgesToUpdate = [];
                        connectedEdges.forEach(function(id) {{
                            edgesToUpdate.push({{id: id, hidden: false}});
                        }});
                        edges.update(edgesToUpdate);
                    }}
                }}
            }});

            // Mostrar aristas iniciales entre temas visibles (si hubiera)
            var visibleEdges = [];
            edges.forEach(function(edge) {{
                var fromNode = nodes.get(edge.from);
                var toNode = nodes.get(edge.to);
                if (!fromNode.hidden && !toNode.hidden) {{
                    visibleEdges.push({{id: edge.id, hidden: false}});
                }}
            }});
            edges.update(visibleEdges);

        </script>
    </body>
    </html>
    """

    with open(HTML_OUTPUT, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"Archivo HTML guardado en: {HTML_OUTPUT}")

## 5. EJECUCIÓN


In [8]:
# 6. EJECUCIÓN
# ------------------------------------------------------
if __name__ == "__main__":
    G = construir_grafo_semantico()
    generar_html(G)
    print("\n¡LISTO! Descarga 'visor_grafo_espanol.html'.")

Cargando datos...
Generando nombres semánticos (y traduciendo)...
Construyendo nodos y traduciendo relaciones...
Grafo construido. Nodos: 586
Generando HTML interactivo...
Archivo HTML guardado en: /content/drive/MyDrive/Proyecto_Tesis_Multimodal/grafo_final/visor_grafo_espanol.html

¡LISTO! Descarga 'visor_grafo_espanol.html'.
